In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Locate the data
data = '../Gold Std.csv'
# Read data with pandas
dataset = pd.read_csv(data)
# Extract the values of the desired column
full_set = dataset['XAU_USD'].values
dataset

,Unnamed: 0,Date,XAU_USD,GOL_Returns_Rolling_Std
0,0,1979-12-28,-0.678952,NaN
1,1,1980-01-01,0.000000,NaN
2,2,1980-01-02,9.326172,NaN
3,3,1980-01-03,13.309513,NaN
4,4,1980-01-04,0.000000,NaN
...,...,...,...,...
10658,10658,2021-02-10,0.280884,0.985257
10659,10659,2021-02-11,-0.915200,0.991293
10660,10660,2021-02-12,-0.104089,0.991185
10661,10661,2021-02-14,-0.128876,0.969552


In [5]:
# We split our data on the 7200 mark as it represents 70% of it
train_data = full_set[5061:9061]
test_data = full_set[9061:]
# dataset.iloc[5061]

In [6]:
# Data Normalization
from sklearn.preprocessing import MinMaxScaler
# Data must be reshaped in order to use fit_transform
training_set = train_data.reshape(-1,1)
test_set = test_data.reshape(-1,1)

sc = MinMaxScaler(feature_range=(0,1))

X_train_scaled = sc.fit_transform(training_set)

In [7]:
# Create lists for x and y train
X_train = []

y_train = []

# We will run our model on 100 timesteps
for i in range(20,4000):
    
    X_train.append(X_train_scaled[i- 20:i,0])

    y_train.append(X_train_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [8]:
# Import the necessary models from keras within tensor flow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [15]:
# Create a callback function with keras, which will monitor thr rmse of each epoch 
# And stop our model on the epoch that offers the best balance so we don't overfit our model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [16]:
# Create the layers of our model droping out and replenishing 20% of our data after each layer
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
# Include a printout of the model summary 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 20, 50)            10400     
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 50)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 50)               

In [17]:
model.compile(optimizer='adam',loss= 'mean_squared_error',
             metrics=[tf.keras.metrics.RootMeanSquaredError()])
# While we git our model we assign it to a variable 
history = model.fit(X_train,y_train,epochs=20,batch_size=100,callbacks=[callback], verbose = 2)

Epoch 1/20
40/40 - 12s - loss: 0.0210 - root_mean_squared_error: 0.1451
Epoch 2/20
40/40 - 2s - loss: 0.0057 - root_mean_squared_error: 0.0756
Epoch 3/20
40/40 - 2s - loss: 0.0053 - root_mean_squared_error: 0.0728
Epoch 4/20
40/40 - 2s - loss: 0.0052 - root_mean_squared_error: 0.0721
Epoch 5/20
40/40 - 2s - loss: 0.0050 - root_mean_squared_error: 0.0705
Epoch 6/20
40/40 - 2s - loss: 0.0051 - root_mean_squared_error: 0.0712
Epoch 7/20
40/40 - 2s - loss: 0.0048 - root_mean_squared_error: 0.0694
Epoch 8/20
40/40 - 2s - loss: 0.0048 - root_mean_squared_error: 0.0690
Epoch 9/20
40/40 - 2s - loss: 0.0048 - root_mean_squared_error: 0.0693
Epoch 10/20
40/40 - 2s - loss: 0.0048 - root_mean_squared_error: 0.0691
Epoch 11/20
40/40 - 2s - loss: 0.0047 - root_mean_squared_error: 0.0684
Epoch 12/20
40/40 - 2s - loss: 0.0047 - root_mean_squared_error: 0.0682
Epoch 13/20
40/40 - 2s - loss: 0.0047 - root_mean_squared_error: 0.0682
Epoch 14/20
40/40 - 2s - loss: 0.0047 - root_mean_squared_error: 0.0686


In [18]:
# Print out the number of epochs used for this model
print(len(history.history['root_mean_squared_error']))

20


In [19]:
# Making Predictions to the test set
dataset_total = np.concatenate((train_data, test_data), axis = 0)
# Prepare our data to the test
inputs = dataset_total[len(dataset_total) - len(test_data) - 100:]
# Reshape our data to fit the upcoming for loop
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

# Create lists to which we'll append x and y values for future test
X_test = []
y_test = []

# Create a loop that will run through groups of 200 timesteps for the test sample
for i in range(20, 1411):
    X_test.append(inputs[i-20:i,0])
    y_test.append(inputs[i, 0])

In [20]:
# Turn our lists into a numpy numbers array
X_test = np.array(X_test)
y_test = np.array(y_test)

# Reshape our array into 3 dimensions so it can fit the LSTM Model
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_test = y_test.reshape(-1, 1)

In [21]:
 model_loss, rmse = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Root Mean Square Error:{rmse}")

44/44 - 3s - loss: 0.0017 - root_mean_squared_error: 0.0409
Normal Neural Network - Loss: 0.0016752950614318252, Root Mean Square Error:0.0409303680062294


In [22]:
predicted_returns = model.predict(X_test)
predicted_returns = sc.inverse_transform(predicted_returns)

In [25]:
predicted_returns

array([[-0.04896753],
       [-0.04995926],
       [-0.05706575],
       ...,
       [-0.01674779],
       [-0.01422229],
       [-0.00649589]], dtype=float32)

In [24]:
y_test = sc.inverse_transform(y_test)


In [26]:
predicted_returns = predicted_returns * 100

In [27]:
import plotly.express as px

In [28]:
output = pd.DataFrame({
    'predicted_returns' : [x[0] for x in predicted_returns],
    'actual_returns' : [x[0] for x in y_test]
})
output

,predicted_returns,actual_returns
0,-4.896753,-1.662394
1,-4.995926,0.250415
2,-5.706575,-0.628019
3,-6.023613,0.614543
4,-5.965167,-1.036372
...,...,...
1386,-1.951439,0.143873
1387,-2.079486,-0.006590
1388,-1.674779,0.767152
1389,-1.422229,1.485343


In [29]:
px.line(output, y=["predicted_returns", "actual_returns"])

In [32]:
output.to_csv('../../Model Results/Gold-Returns-Predicted.csv')
